In [1]:
import os 
import audioop
#import subprocess
from auditok import split, AudioRegion

### Audio Splitter

In [2]:
def SplitSentences(path): 
    
    #PreProcessing
    Audio = AudioRegion.load(path) 
    
    Data = audioop.tomono(bytes(Audio), Audio.sample_width, 0.5, 0.5)
    
    region_mono = AudioRegion(Data, sample_width=Audio.sw, sampling_rate=Audio.sr, channels=1)
    
    regions = region_mono.split(energy_threshold=50, use_channel='mix', min_dur=1, max_dur=15, max_silence = 0.25) 
    
    #Extract pauses
    onset = 0
    pauses = []
    speeches, NewSpeeches = [], []
    for r in regions:
        if onset < r.meta.start:
            pauses.append((onset, r.meta.start))
            onset = r.meta.end
        speeches.append(r)

    #Last pause if there exists
    if onset < Audio.duration:
        pauses.append((onset, Audio.duration))

    silence = []
    for pause in pauses:
        silence.append(Audio.seconds[pause[0]:pause[1]])
        
    #Setting 15 Sec limitation
    i = 0
    max_limit = 15
    while i < len(speeches):
        sound = speeches[i]
        while i < len(speeches)-1 and sound.duration + speeches[i+1].duration < max_limit:
            i += 1
            sound += speeches[i]
        else:
            i+=1
        NewSpeeches.append(sound)
    
    try: os.mkdir("Converted")
    except: pass
  
    i = 0
    #Speech Recognition
    print('Found {0} Audio Sentences: '.format(len(NewSpeeches)))
    for speech in NewSpeeches: 
      
        print("Saving Sentence " + str(i), end="\r") 
    
        filename = speech.save("Converted/Mono " + str(i) + ".wav")
  
        i += 1
  
    print('Audio Sentences processed: ', len(NewSpeeches))

In [5]:
print('Enter the audio file path') 
path = input()
SplitSentences(path)

Enter the audio file path
x1.mp3
Found 217 Audio Sentences: 
Audio Sentences processed:  217
